### Flask - jak udostępnić model dla użytkowników

In [3]:
%%writefile dog_breed_server.py

import os
from sklearn.externals import joblib
from flask import Flask, request, redirect, flash, jsonify
from werkzeug.utils import secure_filename

from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
import pickle

UPLOAD_FOLDER = 'tmp'
ALLOWED_EXTENSIONS = set(['png', 'jpg', 'jpeg', 'gif'])

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

def make_prediction(filename, top=5):
    status = 'SUCCESS'
    
    # ResNEt50 prediction    
    img = image.load_img(filename, target_size=(224, 224))
    X = np.expand_dims(image.img_to_array(img), axis=0)
    X = preprocess_input(X)

    model = ResNet50(weights='imagenet')
    preds = model.predict(X)
    y_pred = decode_predictions(preds, top=top)[0]
    y_pred = { x[1]: "{:.5f}".format(float(x[2])) for x in y_pred}
    
    # Own model prediction
    infile = open('VGG16_224x224x3_D512_D128.model','rb')
    model = pickle.load(infile)
    infile.close()

    infile = open('class_indices','rb')
    class_indices = pickle.load(infile)
    infile.close()

    img = image.load_img(filename, target_size=(224, 224))
    tmp = np.expand_dims(img, axis=0)       
    tmp2 = tmp / 255
    
    own_pred = model.predict_proba(tmp2)
    y_own_pred = {}
    for breed, pred in zip(list(class_indices.keys()),own_pred[0]):
        y_own_pred[breed] = "{:.5f}".format(pred)
            
    return status, y_pred, y_own_pred

def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route('/', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        if 'file' not in request.files:
            flash('No file part')
            return redirect(request.url)
        file = request.files['file']
        if file.filename == '':
            flash('No selected file')
            return redirect(request.url)
        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            full_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
            file.save(full_path)
                    
            status, ResNet50Pred, VGG16_own_model = make_prediction(full_path)
            
            # usuwam zdfdjęcie z serwera po predykcji
            os.remove(full_path)

            return jsonify({
                'filename': filename,
                'ResNet50Pred': ResNet50Pred,
                'VGG16_own_model': VGG16_own_model,
                'status': status
            })
        
    return '''
    <!doctype html>
    <title>Klasyfikacja zdjęć</title>
    <h1>Klasyfikacja zdjęć</h1>
    <p>Wybierz zdjęcie, następnie klikniaj "upload"</p>
    <form method=post enctype=multipart/form-data>
      <p><input type=file name=file>
         <input type=submit value=Upload>
    </form>
    '''

Overwriting dog_breed_server.py


In [ ]:
!FLASK_APP=dog_breed_server.py flask run --port "8895" --host "localhost"

 * Serving Flask app "dog_breed_server.py"
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
Using TensorFlow backend.
 * Running on http://localhost:8895/ (Press CTRL+C to quit)
127.0.0.1 - - [03/May/2019 14:24:07] "GET / HTTP/1.1" 200 -
Instructions for updating:
Colocations handled automatically by placer.
2019-05-03 14:24:14.246366: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX
2019-05-03 14:24:14.268445: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 3399780000 Hz
2019-05-03 14:24:14.269121: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x7f856c4d47e0 executing computations on platform Host. Devices:
2019-05-03 14:24:14.269145: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-05-03 14:24:14.371622: I tensorflow/stream_executor/cuda/cuda_